In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import sys
import matplotlib as mpl
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()    
font_name
plt.rc('font', family=font_name)
mpl.rcParams["axes.unicode_minus"]=False    #마이너스를 문자로 쓰지 않고 숫자로 쓰겠다. 라는 뜻

<font color=red>

## CNN & RNN & CRNN
</font>

    ★★★★★★★★★★★★ RNN ★★★★★★★★★★★★★
    

    (2) RNN(Recurrent Neural Network) : 순환 신경망
        -  현재를 다시 재실행한다?
        - I google at work & I work at google
          → 같은 google이고 같은 work지만 한 문자에서는 동사고 한 문장에서는 명사
          → 각 문장에서 모양이 아얘 동일한데 이 두 문장에서의 의미를 어떻게 구분해낼 것인가
          → 이전의 처리된 데이터를 반영해서 현재의 데이터를 학습시켜주는게 RNN
          
        1) Sequence data
            - 히든계층(cell)에서 빠져나와서 출력된 값이 자기한테 다시 입력이 됨
            - state 출력
                - 옆에있는 레이어(네트워크)에게 출력해서 넘겨주면 옆에 있는 레이어가 값을 입력받아서 또 처리
                - 그 전에 어떤 값이 입력되었는지 고려할 수 있다는 것
                - 옆으로 넘기고 넘기고 넘기고 렛잇고
                - 중간 과정들 중에서 최종 결과는 각각의 h
                - 마지막 과정의 A는 최종 결과 ㅇㅋ (감정구분과 같은 이진 분류로 주로 사용)
예시|1|2|3|...|n|사용예
:---:|:---:|:---:|:---:|:---:|:---:|:---
최종출력|h1|h2|h3|...|hn|다항분류에 주로 쓰이는 결과값
<font color=red><b>state출력</b></font>|A1→|A2→|A3→|...|<b>A</b>|이진분류로 주로 쓰이는 결과값(감정 구분 등)
입력|X1|X2|X3|...|Xn|

        ※ Language model Example
            - hihello 확습하기
            - 입력값 X = h, i, h, e, l, l, o
            - 출력값 y = i, h, e, l, l, o

        ※ POS tagging (품사 구분)
X1=I|X2=work|X3=at|X4=google
:---|:---|:---|:---
tanh─|┐tanh─|┐tanh─|┐tanh
.....↑|↓....↑|↓....↑|↓....↑
A|└→A→|└→A→|└→A
↑|↑|↑|↑
X, b|X, b|X, b|X, b
y1|y2|y3|y4
pronoun<br>0.8|verb<br>0.7|preposition<br>0.6|noun<br>0.8

            - 두번째 처리부터는 'X에 대한 가중치 + 이전에 처리됐던 결과값에 대한 가중치'로 처리
            - 때문에 처리과정을 거칠수록 계속 결과값이 달라짐
        
        2) BPTT_Back Propagation Through Time
            - 시간에 따라서 백프로퍼제이션을 합니당
            
    → Vanila RNN : 특별한 목적이나 특별한 용도로 확장하지 않은 RNN의 기본 알고리즘

<font color=red>

### Vanila RNN
</font>

In [3]:
from tensorflow.contrib import rnn

In [26]:
tf.reset_default_graph()

# 입력 값 준비 : hello (시퀀스 데이터)
# 글자니까 숫자로 바꿔줘야쥬 머신러닝은 숫자바께 모르는 바보니까
## one_hot으로 넣어주께 ㅎ
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

# 셀(cell, 히든계층) 준비
cell = tf.contrib.rnn.BasicRNNCell(num_units=2) # num_units=2 히든계층 갯수

# 입력데이터 준비
x_data = np.array([[h, e, l, l, o]], dtype=np.float32) # RNN은 데이터를 3차원 데이터로 형식을 맞춰줘야 함
print(x_data, "\n→ 5개의 시퀀스 길이가 되는고얌 (X1 = h / X2 = e / ... X5 = o)\n")

# RNN 실행 (static_rnn, dynamic_rnn 등 여러가지)
outputs, _state = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32) # 최종출력&state출력 2개출력을 받아오는 애니까 변수도 2개로 넣어줬음

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(outputs), "\n 셀 준비할때 'num_units=2'으로 히든계층 갯수 2개 지정했으니까 각 2개씩 출력ㅇㅇ")
print("---------------------")
print("맨 마지막 값은 출력값과 마지막state값이 같음~\n", sess.run(_state))

[[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]] 
→ 5개의 시퀀스 길이가 되는고얌 (X1 = h / X2 = e / ... X5 = o)

[[[ 0.6904387   0.60326236]
  [ 0.3204498  -0.9248516 ]
  [ 0.87527156  0.7682024 ]
  [ 0.80973434 -0.46434134]
  [ 0.46368122 -0.5240082 ]]] 
 셀 준비할때 'num_units=2'으로 히든계층 갯수 2개 지정했으니까 각 2개씩 출력ㅇㅇ
---------------------
맨 마지막 값은 출력값과 마지막state값이 같음~
 [[ 0.46368122 -0.5240082 ]]


In [4]:
# 걍 cell 종류 볼라고 찍어봤음 (수업X)
dir(tf.contrib.rnn)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



['AttentionCellWrapper',
 'BasicLSTMCell',
 'BasicRNNCell',
 'BidirectionalGridLSTMCell',
 'CompiledWrapper',
 'Conv1DLSTMCell',
 'Conv2DLSTMCell',
 'Conv3DLSTMCell',
 'ConvLSTMCell',
 'CoupledInputForgetGateLSTMCell',
 'DeviceWrapper',
 'DropoutWrapper',
 'EmbeddingWrapper',
 'FusedRNNCell',
 'FusedRNNCellAdaptor',
 'GLSTMCell',
 'GRUBlockCell',
 'GRUBlockCellV2',
 'GRUCell',
 'GridLSTMCell',
 'HighwayWrapper',
 'IndRNNCell',
 'IndyGRUCell',
 'IndyLSTMCell',
 'InputProjectionWrapper',
 'IntersectionRNNCell',
 'LSTMBlockCell',
 'LSTMBlockFusedCell',
 'LSTMBlockWrapper',
 'LSTMCell',
 'LSTMStateTuple',
 'LayerNormBasicLSTMCell',
 'LayerRNNCell',
 'MultiRNNCell',
 'NASCell',
 'OutputProjectionWrapper',
 'PhasedLSTMCell',
 'RNNCell',
 'ResidualWrapper',
 'SRUCell',
 'TimeFreqLSTMCell',
 'TimeReversedFusedRNN',
 'UGRNNCell',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'best_effort_input_batch_size',
 'stack

<font color=red>

#### < 품사 구분 I google at work & I work at google >
</font>

In [38]:
tf.reset_default_graph()
tf.set_random_seed(777)

# I google at work  = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
# I work at google = [[1, 0, 0, 0], [0, 0, 0, 1],[0, 0, 1, 0], [0, 1, 0, 0]]

# 입력데이터 준비
inputs = np.array([[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]],
                  [[1, 0, 0, 0], [0, 0, 0, 1],[0, 0, 1, 0], [0, 1, 0, 0]]])

tf_inputs = tf.constant(inputs, dtype=tf.float32)

# 셀준비
cell = tf.contrib.rnn.BasicRNNCell(num_units=3) # num_units 히든계층 갯수

# RNN 실행 (static_rnn, dynamic_rnn 등 여러가지)
outputs, _state = tf.nn.dynamic_rnn(cell=cell, inputs=tf_inputs, dtype=tf.float32)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(outputs), "\n 단어 4개 각 'num_units=3'로 히든계층 갯수 3개씩 출력으로 문장 2개ㅇㅇ\n"+
     "각 답(출력)이 다른 이유는 가중치가 히든계층마다 다르니까")
print("---------------------")
print(sess.run(_state), "\n 맨 마지막 값은 출력값과 마지막state값이 같음\n윗줄이 I google at work 아랫줄이 I work at google")
print("---------------------")

# print("I", sess.run(outputs)[0][0])
# print(sess.run(outputs)[0][1])
# print("at", sess.run(outputs)[0][2])
# print(sess.run(outputs)[0][3])

# print("I", sess.run(outputs)[1][0])
# print(sess.run(outputs)[1][1])
# print("at", sess.run(outputs)[1][2])
# print(sess.run(outputs)[1][3])


# variable_names = [v.name for v in tf.trainable_variables()]
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)
"""
선생님 부교재에서 :)
   state  가중치 3행3열
+  입력값 가중치 2행3열
→               5행3열
된거처럼 입력값 가중치랑 state 가중치가 합해져서 7행3열이 된거
"""
sess.close()

[[[-0.50944704  0.33166462  0.6126557 ]
  [-0.20793891  0.24406303 -0.75278705]
  [-0.06346128 -0.52844936  0.68356085]
  [-0.36491966  0.8857268  -0.02324395]]

 [[-0.50944704  0.33166462  0.6126557 ]
  [-0.30707452  0.62735885  0.21719742]
  [ 0.5043804  -0.14038289  0.3744523 ]
  [-0.11641277  0.70696247 -0.7512605 ]]] 
 단어 4개 각 'num_units=3'로 히든계층 갯수 3개씩 출력으로 문장 2개ㅇㅇ
각 답(출력)이 다른 이유는 가중치가 히든계층마다 다르니까
---------------------
[[-0.36491966  0.8857268  -0.02324395]
 [-0.11641277  0.70696247 -0.7512605 ]] 
 맨 마지막 값은 출력값과 마지막state값이 같음
윗줄이 I google at work 아랫줄이 I work at google
---------------------
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


<font color=red>

### hihello 학습과정
</font>

In [63]:
tf.reset_default_graph()
tf.set_random_seed(777)

idx2char = ['h', 'i', 'e', 'l', 'o'] # hihello 중에 중복단어 빼고 숫자화시킬 칭구들

# one_hot encoding
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 0, 1]]]  # o 4

# 입력데이터
x_data = [[0, 1, 0, 2, 3, 3]] # hihell
y_data = [[0, 1, 0, 2, 3, 3, 4]] # hihello
"""
    ※ Language model Example
        - hihello 확습하기
        - 입력값 X = h, i, h, e, l, l
        - 출력값 y = i, h, e, l, l, o
"""

# X, y 변수 준비
num_classes = 5
input_dim = 5 # 입력값 갯수는 h, i, e, l, o 5개
hidden_size = 5 # 입력하는 데이터의 갯수가 몇개로 지정되어있느냐 (= input_dim랑 같음)
batch_size = 1 # 데이터가 작아서 뱃치는 걍 1
sequence_len = 7 # 시퀀스 길이는 7개
learning_rate = 0.1


# 문자열의 shape은 3차원으로 준비하면 됩니다~ [batchsize, sequence length, imput dimension]
X = tf.placeholder(tf.float32, [None, sequence_len,input_dim])
y = tf.placeholder(tf.int32, [None, sequence_len])


# RNN 모델 작성
# 셀(cell, 히든계층) 준비
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

# 초기값 지정
initial_state = cell.zero_state(batch_size, tf.float32)
### ↑↑↑↑↑↑↑↑↑얘 잘못됐나본데...↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑

intial_state
## 셀 안에 어떤 값들이 들어있는지 모르니까 셀에 들어있는 값을 0으로 초기화를 시켜주기 위한 과정
## 이렇게 하면 잡음을 줄일 수 있습니다. 근데 잡음이 뭔데요?

# RNN 실행 (static_rnn, dynamic_rnn 등 여러가지)
outputs, _state = tf.nn.dynamic_rnn(cell=cell, inputs=X, dtype=tf.float32, initial_state=initial_state)

# 2차원으로 평면화 (FC를 위해서)
X_for_fc = tf.reshape(outputs, [-1, hidden_size])

# FC
# FC를 위한 W, b, logit을 한번에 한줄에 해결해주는 함수가 있습니다 :)
## tf.contrib.layers.fully_connected(입력값, 출력갯수, activationfunction)
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_outputs=num_classes, activation_fn=None)

# 비용계산을 위한 차원 맞추기
outputs = tf.reshape(outputs, [batch_size, sequence_len, num_classes])
weights = tf.ones([batch_size, sequence_len])

# 비용계산_시퀀스를 위한 비용계산 함수↓↓
## tf.contrib.seq2seq.sequence_loss(logit 너 3차원ㅋ, target, weights)
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y, weights=weights)
loss = tf.reduce_mean(sequence_loss)

# 훈련시키기
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)


# 예측값
prediction = tf.argmax(outputs, 2)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        _, cost = sess.run([train, loss], feed_dict={X:x_one_hot, y:y_data})
        result = sess.run(prediction, feed_dict={X:x_one_hot})
        print(i, "loss : ", cost, "    prediction : ", result, "    true Y", y_data)
    result_str = [idx2char[c] for c in np.squeeze(result)] # 인덱스 값으로 뽑아와서 문자로 변환할거얌
    # np.squeeze : 차원을 줄여주는 함수
    print("\n결과 : ", ".".join(result_str))
    

0 loss :  1.638345     prediction :  [[0 3 3 3 3 3 2]]     true Y [[0, 1, 0, 2, 3, 3, 4]]
1 loss :  1.386576     prediction :  [[0 0 0 3 3 3 4]]     true Y [[0, 1, 0, 2, 3, 3, 4]]
2 loss :  1.1635329     prediction :  [[0 0 0 3 3 3 4]]     true Y [[0, 1, 0, 2, 3, 3, 4]]
3 loss :  0.93181574     prediction :  [[0 1 0 2 3 3 4]]     true Y [[0, 1, 0, 2, 3, 3, 4]]
4 loss :  0.70485556     prediction :  [[0 1 0 2 3 3 4]]     true Y [[0, 1, 0, 2, 3, 3, 4]]
5 loss :  0.50350326     prediction :  [[0 1 0 2 3 3 4]]     true Y [[0, 1, 0, 2, 3, 3, 4]]
6 loss :  0.3518529     prediction :  [[0 1 0 2 3 3 4]]     true Y [[0, 1, 0, 2, 3, 3, 4]]
7 loss :  0.2512133     prediction :  [[0 1 0 2 3 3 4]]     true Y [[0, 1, 0, 2, 3, 3, 4]]
8 loss :  0.18412565     prediction :  [[0 1 0 2 3 3 4]]     true Y [[0, 1, 0, 2, 3, 3, 4]]
9 loss :  0.13794506     prediction :  [[0 1 0 2 3 3 4]]     true Y [[0, 1, 0, 2, 3, 3, 4]]
10 loss :  0.10591277     prediction :  [[0 1 0 2 3 3 4]]     true Y [[0, 1, 0, 2, 3, 3

<font color=red>

### LSTM
</font>

    - Vanila RNN의 문제점
        - gradient vanishing
            - 값이 너무 작아져서 오차를 못찾아염
        - gradient exploding
            - ReLU같은거 사용해서 값을 또 너무 키워버리면 최저비용에 도달을 못해
    - 이런 것들을 보완해서 나온게 LSTM
    - 바닐라와의 차이점은 state가 바닐라는 하나 LSTM은 두개
    

In [64]:
# ???? 선생님이 주신 코드

tf.reset_default_graph()
tf.set_random_seed(777)

inputs = np.array([[[1, 0]]])

tf_inputs = tf.constant(inputs, dtype=tf.float32)
cell = tf.nn.rnn_cell.LSTMCell(num_units=1)
outputs, _states = tf.nn.dynamic_rnn(cell=cell, dtype=tf.float32, inputs=tf_inputs)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_run, states_run = sess.run([outputs, _states])
    print("output values")
    print(outputs_run)
    print("\nmemory cell value ")
    print(states_run.c)
    print("\nhidden state value ")
    print(states_run.h)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
output values
[[[0.09927537]]]

memory cell value 
[[0.18134572]]

hidden state value 
[[0.09927537]]


<font color=red>

### LSTM을 이용한 문장 분류
</font>

In [81]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the thiadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]

df = pd.DataFrame(paragraph_dict_list)
df = df[["paragraph", "category"]]
df

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food
5,the garlic fries were a great starter (and a h...,food
6,our meal was excellent i had the pasta ai form...,food
7,what i enjoy most about palo alto is so many r...,food
8,the drinks came out fairly quickly a good two ...,food
9,despite the not so good burger the service was...,food


In [82]:
# 중복된 데이터 제거하기
results = set()
df["paragraph"].str.lower().str.split().apply(results.update)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: paragraph, dtype: object

In [83]:
# 순서대로 인덱스를 붙여서 각 단어를 숫자로 바꿔주기
idx2word = dict(enumerate(results))
idx2word[444]

'seating'

In [84]:
word2idx =  {v:k for k, v in idx2word.items()} # 키와 값의 자리를 바꿔서 반대 의미로 사용할 수 있게 만들음

In [85]:
word2idx["bread"]

349

In [86]:
# 선생님 코드

# 각 문장을 숫자로 바꿔주는 함수
def encode_paragraph(paragraph):
    words = paragraph.split(" ")
    encoded = []
    for word in words:
        encoded.append([word2idx[word]])
    
    return encoded

# 카테고리를 숫자로 바꿔주는 함수
def encode_category(category):
    if category == "food":
        return [1, 0]
    else:
        return [0, 1]

# 전체 단어 개수 카운트 함수
def word_cnt(paragraph):
    return len(paragraph.split(" "))

In [87]:
df["enc_paragraph"] = df.paragraph.apply(encode_paragraph)

In [88]:
df["enc_category"] = df.category.apply(encode_category)

In [89]:
df["seq_length"] = df.paragraph.apply(word_cnt)

이전까지 샘플들은 입력되는 데이터의 길이가 다 같았지만   
실제 데이터들은 들쑥날쑥와리가리   
긴 애들은 그렇다 치는데 짧은 애들은 빈 공간이 전부 0으로 처리되면 그것도 하나의 데이터로 인식되는데   
그걸 해결하기 위해서~~ **dynamic_rnn**을 쓰는거임   
→ 일단 가장 긴 길이인 애를 기준으로 크기를 잡아주고   
   그보다 작은 애들은 패딩으로 표시를 해줌   
   그러면 다이나믹알앤앤이 그 표시를 보고 그 부분으로는 훈련하지 않게 됨   

In [91]:
# 그니까 일단 제일 긴놈이 누군지 알아보자
# 문장의 최대 길이 알아보기 위한 선생님 코드
max_word_cnt = 0
for row in df["paragraph"]:
    if len(row.split(" ")) > max_word_cnt:
        max_word_cnt = len(row.split(" "))
        
print(max_word_cnt) # > 91개

91


In [92]:
# 문장의 길이가 91개가 안될 경우 -1로 패딩처리

def sequence_padding(enc_paragraph):
    seq_length = len(enc_paragraph)
    for i in range(seq_length, max_word_cnt):
        enc_paragraph.append([-1])
        
    return enc_paragraph

In [93]:
df["enc_paragraph"] = df.enc_paragraph.apply(sequence_padding)

In [94]:
print(df["enc_paragraph"][0])

[[14], [304], [22], [25], [173], [359], [182], [304], [429], [339], [238], [184], [358], [303], [202], [37], [361], [467], [292], [167], [187], [239], [13], [389], [414], [193], [136], [387], [467], [12], [103], [309], [268], [90], [80], [458], [179], [131], [483], [125], [337], [136], [40], [379], [17], [333], [501], [99], [294], [238], [520], [467], [269], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]


In [95]:
# 입력값을 배열로 변환
enc_paragraph = np.array(df.enc_paragraph.tolist())
enc_category = np.array(df.enc_category.tolist())
seq_length = np.array(df.seq_length.tolist())

In [98]:
# 데이터 준비
train_X = enc_paragraph # shape → (20, 91, 1)
train_y = enc_category # shape → (20, 2)

In [100]:
# 모델 구축 (선생님 코드)

tf.reset_default_graph()
tf.set_random_seed(777)

learning_rate = 0.001
n_epochs = 300

X = tf.placeholder(tf.float32, [None, max_word_cnt, 1])
y = tf.placeholder(tf.int32, [None, 2])

embedding = tf.layers.dense(X, 5)

cell = tf.nn.rnn_cell.LSTMCell(num_units=64)
output, state = tf.nn.dynamic_rnn(cell, embedding, dtype=tf.float32, sequence_length=seq_length)

dense_layer = tf.layers.dense(state.h, 32) # dense layer는 FC를 좀더 간단하게 쓰기위해서 사용되었다고 말씀하심
logits = tf.layers.dense(dense_layer, 2) # 근데 뭔소린지 1도 모르겠음
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

"""
emdedding 작업?
FC 과정을 한번에 처리해주는게 dense라고?
??????? 의문투성이
"""

'\nemdedding 작업?\nFC 과정을 한번에 처리해주는게 dense라고?\n??????? 의문투성이\n'

In [104]:
print(X)
print(embedding)
print(state)
print(dense_layer)
print(logits)

Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, 91, 5), dtype=float32)
LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 64) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 64) dtype=float32>)
Tensor("dense_1/BiasAdd:0", shape=(?, 32), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 2), dtype=float32)


In [105]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        _, cost = sess.run([train, loss], feed_dict={X:train_X, y:train_y})
        if epoch % 50 == 0:
            pred = tf.nn.softmax(logits)
            correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_pred, "float"))
            cur_acc = accuracy.eval({X:train_X, y:train_y})
            print("epoch:" + str(epoch) + ", loss:" + str(cost) + ", accuracy:" + str(cur_acc))

epoch:0, loss:0.86405915, accuracy:0.5
epoch:50, loss:0.42940608, accuracy:0.75
epoch:100, loss:0.20808855, accuracy:0.9
epoch:150, loss:0.020990973, accuracy:1.0
epoch:200, loss:0.0037455582, accuracy:1.0
epoch:250, loss:0.0010593401, accuracy:1.0


In [ ]:
<font color=red>

### LSTM RNN(얘가 바닐라보다 더 자주쓰임), Multistack RNN
</font>

In [ ]:
<font color=red>

### < 뀨 >
</font>